In [1]:
%reload_ext autoreload
%autoreload 2

! [ ! -L /datasets ] && ln -s /data/datasets/cv /datasets

from k12libs.utils.nb_easy import k12ai_set_notebook

k12ai_set_notebook(cellw=95)

## 需掌握知识点

1. 激活函数层(Relu, Sigmoid)
2. Dropout层
3. Batchnorm层

In [2]:
%reset -f

from pyr.app.k12ai import EasyaiClassifier, EasyaiTrainer
import torch
from torch import nn
from sklearn.metrics import confusion_matrix

class CustomClassifier(EasyaiClassifier):
    
    def __init__(self):
        super().__init__()
        self.input_size = 224 # 输入模型的图片大小
        self.output_size = 5  # 模型输出大小(分类数)
        self.batch_size = 32  # 输入模型的图片数量
        
        # 调试: 日志输出模型shape
        self.example_input_array = torch.zeros(
            self.batch_size, 3, self.input_size, self.input_size) 

    def prepare_dataset(self):
        """
        flowers5: 5个分类
            0: 'rose'        玫瑰
            1: 'daisy'       雏菊
            2: 'sunflower'   向日葵
            3: 'dandelion'   蒲公英
            4: 'tulip'       郁金香
        """

        return self.load_flowers5()

    def build_model(self):
        class NeuralNet(nn.Module):
            def __init__(self, num_classes):
                super().__init__()
                self.features = nn.Sequential(
                    nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(kernel_size=2),
                    nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(kernel_size=2),
                    nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3),
                    nn.ReLU(inplace=True),
                    nn.Dropout2d(),
                    nn.MaxPool2d(kernel_size=2),
                    nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
                    nn.ReLU(inplace=True),
                    nn.MaxPool2d(kernel_size=2),
                    nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3),
                    nn.ReLU(inplace=True),
                    nn.Dropout2d(0.6),
                    nn.MaxPool2d(kernel_size=2),
                )
                self.classifier = nn.Sequential(
                    nn.Flatten(start_dim=1, end_dim=-1),
                    nn.Dropout(p=0.4),
                    nn.Linear(in_features=3200, out_features=512),
                    nn.ReLU(inplace=True),
                    nn.Dropout(p=0.4),
                    nn.Linear(in_features=512, out_features=num_classes)
                  )
                
            def forward(self, x):     
                return self.classifier(self.features(x))
            
        return NeuralNet(num_classes=self.output_size) # input shape: (32, 3, 224, 224)
    
    def train_dataloader(self):
        # data_augment = [
        #     self.random_rotation(degrees=30),
        #     self.random_horizontal_flip(),
        #     self.random_vertical_flip()
        # ]
        return self.get_dataloader('train', self.batch_size, self.input_size, normalize=True, shuffle=False, data_augment=None)
     
    def val_dataloader(self):
        return self.get_dataloader('val', self.batch_size, self.input_size, normalize=True)
    
    def test_dataloader(self):
        return self.get_dataloader('test', self.batch_size, self.input_size)
    
    def training_step(self, batch, batch_idx):
        x, y_true, _ = batch
        y_pred = self.forward(x)
        loss = self.cross_entropy(y_pred, y_true)
        # print(torch.argmax(y_pred, axis=1).flatten())
        acc = (torch.argmax(y_pred, axis=1) == y_true).float().mean()
        log = {'loss': loss, 'acc': acc}
        return log
    
    def validation_step(self, batch, batch_idx):
        x, y_true, _ = batch
        y_pred = self.forward(x)
        loss = self.cross_entropy(y_pred, y_true)
        acc = (torch.argmax(y_pred, axis=1) == y_true).float().mean()
        log = {'val_loss': loss, 'val_acc': acc}
        return log

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        log = {'val_loss': avg_loss, 'val_acc': avg_acc}
        return {'progress_bar': log}
    
    def configure_optimizer(self, model):
        """
        优化器:
            adam: 利用梯度的一阶矩估计和二阶矩估计动态调整每个参数的学习率
            sgd:
        """
        # 优化算法
        return self.adam(model.parameters(), base_lr=0.001, weight_decay=0.0001)
        # return self.sgd(model.parameters(), base_lr=0.01, momentum=0.88, nesterov=False, weight_decay=0.001)
    
    def configure_scheduler(self, optimizer):
        """
        学习率策略:
            step_lr: 每step_size个epoch，lr会自动乘以gamma(lr = lr*gamma)
            multistep_lr: 按照milestones里的值,阶段地修改学习率(lr = lr*gamma)
            reduceon_lr: 监控val_loss的值, 如果在容忍patience值指定的epoch次数内没有减少, 则修改学习率(lr = lr*factor)
        """
        # return self.step_lr(optimizer, step_size=30, gamma=0.1)
        # return self.multistep_lr(optimizer, milestones=[5, 10], gamma=0.1)
        return self.reduceon_lr(optimizer, factor=0.1, patience=4, min_lr=0.000001)
    
    
trainer = EasyaiTrainer(
    max_epochs=200,
    log_lr=True,
    model_summary='full',
    early_stop={'monitor': 'val_acc', 'patience': 6, 'mode': 'max'},
    # early_stop={'monitor': 'val_loss', 'patience': 6, 'mode': 'min'},
    ckpt_path='9-2')

model = CustomClassifier()

# 训练
trainer.fit(model)

# 评估
trainer.test()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


--------------------------------------------------------------------------------
{'label_names': ['rose', 'daisy', 'sunflower', 'dandelion', 'tulip'],
 'mean': [0.4588, 0.4199, 0.3005],
 'num_classes': 5,
 'num_records': 4323,
 'std': [0.2477, 0.2201, 0.2253]}

--------------------------------------------------------------------------------

   | Name               | Type       | Params | In sizes           | Out sizes         
---------------------------------------------------------------------------------------------
0  | model              | NeuralNet  | 2 M    | [32, 3, 224, 224]  | [32, 5]           
1  | model.features     | Sequential | 388 K  | [32, 3, 224, 224]  | [32, 128, 5, 5]   
2  | model.features.0   | Conv2d     | 896    | [32, 3, 224, 224]  | [32, 32, 222, 222]
3  | model.features.1   | ReLU       | 0      | [32, 32, 222, 222] | [32, 32, 222, 222]
4  | model.features.2   | MaxPool2d  | 0      | [32, 32, 222, 222] | [32, 32, 111, 111]
5  | model.features.3   | Conv2d  

Saving latest checkpoint..
Epoch 00046: early stopping triggered.



(fit)	GPU-0 memory allocated: 16.49 MB	 max memory allocated: 833.08 MB
--------------------------------------------------------------------------------
{'label_names': ['rose', 'daisy', 'sunflower', 'dandelion', 'tulip'],
 'mean': [0.4588, 0.4199, 0.3005],
 'num_classes': 5,
 'num_records': 4323,
 'std': [0.2477, 0.2201, 0.2253]}



(fit)	GPU-0 memory allocated: 24.49 MB	 max memory allocated: 833.08 MB
(test)	GPU-0 memory allocated: 24.49 MB	 max memory allocated: 833.08 MB
--------------------------------------------------------------------------------
{'test_acc': 0.2366071492433548}
--------------------------------------------------------------------------------
